In [1]:
#Install library
!pip install -q "great-expectations==0.18.19"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 35.4 MB/s eta 0:00:00


In [4]:
#Membuat file gx
from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='./')

In [5]:
from great_expectations.core.batch import BatchRequest
from great_expectations.data_context import DataContext
from pyspark.sql import SparkSession

#Inisialisasi SparkSession
spark = SparkSession.builder.appName("great_expectations").getOrCreate()

#Inisialisasi Great Expectations context
context = DataContext()

  warnings.warn(



In [ ]:
#Membaca data ke dalam Spark DataFrames
df_dim_country = spark.read.csv('/content/dim_country.csv', header=True, inferSchema=True)
df_fact_education = spark.read.csv('/content/fact_education.csv', header=True, inferSchema=True)
df_fact_gdp = spark.read.csv('/content/fact_gdp.csv', header=True, inferSchema=True)
df_fact_homi = spark.read.csv('/content/fact_homicide.csv', header=True, inferSchema=True)

#Register Spark DataFrames sebagai sumber data
dim_country = context.sources.add_spark("spark-country-data")
fact_education = context.sources.add_spark("spark-edu-data")
fact_gdp = context.sources.add_spark("spark-gdp-data")
fact_homicide = context.sources.add_spark("spark-homicide-data")

#Tambahkan assets untuk setiap sumber data
asset_dim_country = dim_country.add_dataframe_asset(name="Country-data", dataframe=df_dim_country)
asset_fact_education = fact_education.add_dataframe_asset(name="Education-data", dataframe=df_fact_education)
asset_fact_gdp = fact_gdp.add_dataframe_asset(name="GDP-data", dataframe=df_fact_gdp)
asset_fact_homicide = fact_homicide.add_dataframe_asset(name="Homicide-data", dataframe=df_fact_homi)

#Buat batch requests untuk setiap asset
batch_request_country = asset_dim_country.build_batch_request()
batch_request_edu = asset_fact_education.build_batch_request()
batch_request_gdp = asset_fact_gdp.build_batch_request()
batch_request_homi = asset_fact_homicide.build_batch_request()

In [ ]:
#Buat suite untuk tiap dataset
dim_country_suite = 'expectation-dim-country-dataset'
fact_edu_suite = 'expectation-fact-edu-dataset'
fact_gdp_suite = 'expectation-fact-gdp-dataset'
fact_homi_suite = 'expectation-fact-homi-dataset'

#Menambahkan expectation suite
context.add_or_update_expectation_suite(dim_country_suite)
context.add_or_update_expectation_suite(fact_edu_suite)
context.add_or_update_expectation_suite(fact_gdp_suite)
context.add_or_update_expectation_suite(fact_homi_suite)

#Dim country dataset
validator_dimcountry = context.get_validator(
    batch_request=batch_request_country,
    expectation_suite_name=dim_country_suite
)

#Fact edu dataset
validator_edu = context.get_validator(
    batch_request=batch_request_edu,
    expectation_suite_name=fact_edu_suite
)

#Fact gdp dataset
validator_gdp = context.get_validator(
    batch_request=batch_request_gdp,
    expectation_suite_name=fact_gdp_suite
)

#Fact homi dataset
validator_homi = context.get_validator(
    batch_request=batch_request_homi,
    expectation_suite_name=fact_homi_suite
)

Expectation untuk tabel dim country

In [8]:

validator_dimcountry.expect_column_to_exist("country_status")

  warnings.warn(



Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_to_exist",
    "kwargs": {
      "column": "country_status",
      "batch_id": "spark-country-data-Country-data"
    },
    "meta": {}
  },
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [9]:
validator_dimcountry.expect_column_value_lengths_to_be_between("country", min_value=2)

Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_value_lengths_to_be_between",
    "kwargs": {
      "min_value": 2,
      "column": "country",
      "batch_id": "spark-country-data-Country-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 202,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [10]:
validator_dimcountry.expect_column_values_to_be_in_set('country_status', ['High Income', 'Middle Income', 'Low Income', 'Conflict Zone'])

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "column": "country_status",
      "value_set": [
        "High Income",
        "Middle Income",
        "Low Income",
        "Conflict Zone"
      ],
      "batch_id": "spark-country-data-Country-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 202,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 36,
    "missing_percent": 17.82178217821782,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Expectation untuk fact edu

In [11]:
validator_edu.expect_column_to_exist("country_year")

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_to_exist",
    "kwargs": {
      "column": "country_year",
      "batch_id": "spark-edu-data-Education-data"
    },
    "meta": {}
  },
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [13]:
validator_edu.expect_column_values_to_not_be_null('country')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "country",
      "batch_id": "spark-edu-data-Education-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 202,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [14]:
validator_edu.expect_column_values_to_not_be_null('completion_rate')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "completion_rate",
      "batch_id": "spark-edu-data-Education-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 202,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [15]:
validator_edu.expect_column_values_to_not_be_null('dropout_rate')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "dropout_rate",
      "batch_id": "spark-edu-data-Education-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 202,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [16]:
validator_edu.expect_column_values_to_not_be_null('unemployment_rate')

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "unemployment_rate",
      "batch_id": "spark-edu-data-Education-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 202,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Expectation untuk fact gdp

In [17]:
validator_gdp.expect_column_to_exist("country_year")

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_to_exist",
    "kwargs": {
      "column": "country_year",
      "batch_id": "spark-gdp-data-GDP-data"
    },
    "meta": {}
  },
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [18]:
validator_gdp.expect_column_values_to_not_be_null("country_year")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "country_year",
      "batch_id": "spark-gdp-data-GDP-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 17024,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [19]:
validator_gdp.expect_column_values_to_not_be_null("year")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "year",
      "batch_id": "spark-gdp-data-GDP-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 17024,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Expectation untuk fact homicide

In [23]:
validator_homi.expect_column_to_exist("country_year")

Calculating Metrics:   0%|          | 0/2 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_to_exist",
    "kwargs": {
      "column": "country_year",
      "batch_id": "spark-homicide-data-Homicide-data"
    },
    "meta": {}
  },
  "result": {},
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [24]:
validator_homi.expect_column_values_to_not_be_null("country_year")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_not_be_null",
    "kwargs": {
      "column": "country_year",
      "batch_id": "spark-homicide-data-Homicide-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 1306,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [25]:
validator_homi.expect_column_values_to_be_unique("country_year")

Calculating Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_unique",
    "kwargs": {
      "column": "country_year",
      "batch_id": "spark-homicide-data-Homicide-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 1306,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [26]:
validator_homi.expect_column_value_lengths_to_be_between('number_convicted_for_intentional_homicide', 1)

Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_value_lengths_to_be_between",
    "kwargs": {
      "column": "number_convicted_for_intentional_homicide",
      "min_value": 1,
      "batch_id": "spark-homicide-data-Homicide-data"
    },
    "meta": {}
  },
  "result": {
    "element_count": 1306,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}